In [27]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
	temperature=0.1, 
	streaming=True,
	callbacks=[
		StreamingStdOutCallbackHandler()
	]
)

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}?",)
prompt = template.format(country_a="Mexico", country_b="Thailand")
chat.predict(prompt)

The distance between Mexico and Thailand is approximately 9,500 miles (15,300 kilometers) when measured in a straight line.

'The distance between Mexico and Thailand is approximately 9,500 miles (15,300 kilometers) when measured in a straight line.'

In [28]:
template2 = ChatPromptTemplate.from_messages(
	[
		("system", "You are a geography expert. And you only reply in {language}."),
		("ai", "Ciao, mi chiamo {name}!"),
		("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?"),
	]
)

prompt2 = template2.format_messages(
	language="Greek",
	name="Socrates",
	country_a="Mexico",
	country_b="Thailand",
)

prompt2
chat.predict_messages(prompt2)

Γεια σας! Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα. Το όνομά μου είναι Σωκράτης. Πώς μπορώ να βοηθήσω;

AIMessageChunk(content='Γεια σας! Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα. Το όνομά μου είναι Σωκράτης. Πώς μπορώ να βοηθήσω;')

In [29]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
	
	def parse(self, text):
		items = text.strip().split(",")
		return list(map(str.strip, items))


p = CommaOutputParser()
p.parse("Hello, how, are, you?")

['Hello', 'how', 'are', 'you?']

In [30]:
# bad request

template3 = ChatPromptTemplate.from_messages(
	[
		(
			"system", 
			"You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else."
		),
		(
			"human",
			"{question}",
		),
	]
)

prompt3 = template3.format_messages(
	max_items=10,
	question="What are the colors?",
)

result = chat.predict_messages(prompt3)

p = CommaOutputParser()
p.parse(result.content)

red, blue, green, yellow, orange, purple, pink, black, white, brown

['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [31]:
# good request with chain

chain = template3 | chat | CommaOutputParser()
chain.invoke({
	"max_items": 5,
	"question": "what are the pokemons?"
})

pikachu, charmander, bulbasaur, squirtle, jigglypuff

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']

In [32]:
chef_prompt = ChatPromptTemplate.from_messages([
	(
		"system",
		"You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."
	),
	(
		"human",
		"I want to cook {cuisine} food." 
	),
])

chef_chain = chef_prompt | chat


veg_chef_prompt = ChatPromptTemplate.from_messages([
	(
		"system",
		"You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain thier preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."
	),
	(
		"human",
		"{recipe}" 
	),
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe" : chef_chain} | veg_chain

final_chain.invoke({
	"cuisine" : "indian",
})

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic and popular dish - Chicken Tikka Masala. Here's a simple recipe for you to try at home:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 tablespoons vegetable oil
- 2 tablespoons garam masala
- 1 tablespoon ground cumin
- 1 tablespoon ground coriander
- 1 teaspoon turmeric
- 1 teaspoon paprika
- 1 teaspoon chili powder (adjust to taste)
- Salt and pepper to taste
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1-inch piece of ginger, grated
- 1 can (14 oz) crushed tomatoes
- 1 cup heavy cream
- Fresh cilantro, chopped (for garnish)

Instructions:
1. In a bowl, mix together the yogurt, lemon juice, 1 tablespoon of vegetable oil, garam masala, cumin, coriander, turmeric, paprika, chili powder, salt, and pepper. Add the chicken pieces and coat them well with the marinade. Cover and refrigera

AIMessageChunk(content="For a vegetarian version of Chicken Tikka Masala, you can easily substitute the chicken with a plant-based alternative such as tofu or paneer. Here's how you can prepare these alternatives:\n\n1. **Tofu**: \n   - Use firm or extra-firm tofu for this recipe. Press the tofu to remove excess water by wrapping it in a clean kitchen towel and placing a heavy object on top for about 15-20 minutes.\n   - Cut the tofu into bite-sized cubes and follow the same marinating process as you would with the chicken. Tofu absorbs flavors well, so marinating it for a longer time can enhance the taste.\n   - Instead of baking, you can pan-fry the marinated tofu in a bit of oil until it's golden brown and slightly crispy.\n\n2. **Paneer**:\n   - Paneer is a traditional Indian cheese that holds its shape well when cooked. You can find it in Indian grocery stores or make it at home by curdling hot milk with lemon juice or vinegar.\n   - Cut the paneer into cubes and lightly fry them 